In [ ]:
import zipfile as zf
file_to_extract = "archive (30).zip"
with zf.ZipFile(file_to_extract, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import zipfile as zf
file_to_extract = "archive (31).zip"
with zf.ZipFile(file_to_extract, 'r') as zip_ref:
    zip_ref.extractall()

In [ ]:
import keras
import tensorflow as tf
from keras.models import load_model
from keras.models import Sequential , Model
from tensorflow.keras import layers , models
from keras import optimizers
from keras.layers import Dense, Dropout, Flatten , BatchNormalization , GlobalAveragePooling2D
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.applications.resnet import ResNet101
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint
from tensorflow.keras.models import load_model

In [ ]:
data_dir = "/content/brain_tumor_dataset"
classes = ["yes", "no"]

In [ ]:
def preprocess_for_resnet(img, size=224):
    img = cv2.resize(img, (size, size))
    img = np.repeat(img[..., np.newaxis], 3, axis=-1)
    return preprocess_input(img)

In [ ]:
X = []
y = []
for label, cls in enumerate(classes):
    folder = os.path.join(data_dir, cls)
    for file in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, file), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            X.append(preprocess_for_resnet(img))
            y.append(label)

X = np.array(X)
y = np.array(y)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)


In [ ]:
train_data_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
train_generator = train_data_gen.flow(X_train, y_train, batch_size=32)

In [ ]:
val_datagen = ImageDataGenerator()
val_generator = val_datagen.flow(X_val, y_val, batch_size=32, shuffle=False)

In [ ]:
from keras.layers import Input
input_tensor = Input(shape=(224, 224, 3))

In [ ]:
resnet_model = ResNet101(weights='imagenet', include_top=False,input_tensor=input_tensor)

In [ ]:
for layer in resnet_model.layers:
    layer.trainable = False

In [ ]:
x = GlobalAveragePooling2D()(resnet_model.output)
x = Dense(64, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(32, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)

model = Model(inputs=input_tensor, outputs=output)

In [ ]:
callbacks_classify=[
    EarlyStopping(patience=3, restore_best_weights=True,monitor="val_loss"),
    ModelCheckpoint("best_model_classification.keras", save_best_only=True,monitor="val_accuracy")
]

In [ ]:
model.compile(optimizer=optimizers.Adam(1e-4),
              loss="binary_crossentropy",
              metrics=["accuracy"])


In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=callbacks_classify
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 47s 4s/step - accuracy: 0.4654 - loss: 0.7829 - val_accuracy: 0.6471 - val_loss: 0.6540
Epoch 2/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 397ms/step - accuracy: 0.6294 - loss: 0.6297 - val_accuracy: 0.6275 - val_loss: 0.6145
Epoch 3/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 492ms/step - accuracy: 0.6741 - loss: 0.5744 - val_accuracy: 0.6471 - val_loss: 0.5331
Epoch 4/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 631ms/step - accuracy: 0.7157 - loss: 0.5421 - val_accuracy: 0.7451 - val_loss: 0.4670
Epoch 5/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 659ms/step - accuracy: 0.7791 - loss: 0.4729 - val_accuracy: 0.9216 - val_loss: 0.4168
Epoch 6/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 518ms/step - accuracy: 0.8143 - loss: 0.4719 - val_accuracy: 0.9216 - val_loss: 0.3805
Epoch 7/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 408ms/step - accuracy: 0.8571 - loss: 0.4393 - val_accuracy: 0.9216 - val_loss: 0.3448
Epoch 8/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 402ms/step - accuracy: 0.8613 - loss: 0.4009 - val_accuracy: 0.9216 - val_loss: 0

In [ ]:
pip install segmentation-models-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
def unet_model(input_size=(128, 128, 3)):
    inputs = layers.Input(input_size)

    c1 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D((2,2))(c1)

    c2 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D((2,2))(c2)

    c3 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D((2,2))(c3)

    c4 = layers.Conv2D(512, (3,3), activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, (3,3), activation='relu', padding='same')(c4)

    u5 = layers.Conv2DTranspose(256, (2,2), strides=(2,2), padding='same')(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, (3,3), activation='relu', padding='same')(c5)

    u6 = layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding='same')(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, (3,3), activation='relu', padding='same')(c6)

    u7 = layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding='same')(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, (3,3), activation='relu', padding='same')(c7)

    outputs = layers.Conv2D(1, (1,1), activation='sigmoid')(c7)

    model = models.Model(inputs, outputs)
    return model

In [ ]:
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │      1,792 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │     36,928 │ conv2d[0][0]      │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d[0]… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │    147,584 │ conv2d_2[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │    295,168 │ max_pooling2d_1[… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │    590,080 │ conv2d_4[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │  1,180,160 │ max_pooling2d_2[… │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16,    │  2,359,808 │ conv2d_6[0][0]    │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose    │ (None, 32, 32,    │    524,544 │ conv2d_7[0][0]    │
│ (Conv2DTranspose)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 32, 32,    │          0 │ conv2d_transpose… │
│ (Concatenate)       │ 512)              │            │ conv2d_5[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_8 (Conv2D)   │ (None, 32, 32,    │  1,179,904 │ concatenate[0][0] │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 32, 32,    │    590,080 │ conv2d_8[0][0]    │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_transpose_1  │ (None, 64, 64,    │    131,200 │ conv2d_9[0][0]  

 Total params: 7,697,345 (29.36 MB)

 Trainable params: 7,697,345 (29.36 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
data_dir = "/content/kaggle_3m"
patients = os.listdir(data_dir)
print(f"Total patients: {len(patients)}")
patients = [p for p in patients if os.path.isdir(os.path.join(data_dir, p))]
print(f"Total patients: {len(patients)}")


Total patients: 112
Total patients: 110


In [ ]:
np.random.seed(42)

np.random.shuffle(patients)

n_total = len(patients)
n_train = int(0.7 * n_total)
n_val = int(0.10 * n_total)

train_patients = patients[:n_train]
val_patients = patients[n_train:n_train+n_val]
test_patients = patients[n_train+n_val:]

print("Train:", len(train_patients))
print("Val:", len(val_patients))
print("Test:", len(test_patients))


Train: 77
Val: 11
Test: 22


In [ ]:
def get_images_from_patients(patient_list, base_dir):
    images, masks = [], []
    for patient in patient_list:
        patient_path = os.path.join(base_dir, patient)
        for fname in os.listdir(patient_path):
            if fname.endswith(".tif") and "_mask" not in fname:
                img_path = os.path.join(patient_path, fname)
                mask_path = img_path.replace(".tif", "_mask.tif")
                images.append(img_path)
                masks.append(mask_path)
    return images, masks

train_imgs, train_masks = get_images_from_patients(train_patients, data_dir)
val_imgs, val_masks = get_images_from_patients(val_patients, data_dir)
test_imgs, test_masks = get_images_from_patients(test_patients, data_dir)

print("Train slices:", len(train_imgs))
print("Val slices:", len(val_imgs))
print("Test slices:", len(test_imgs))


Train slices: 2824
Val slices: 386
Test slices: 719


In [ ]:
def preprocess_mask(mask, IMG_SIZE=128):
    mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))
    mask = cv2.equalizeHist(mask)
    mask = (mask > 10).astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask


In [ ]:
IMG_SIZE = 128
def preprocess_mask(mask, IMG_SIZE=128):
    mask = cv2.resize(mask, (IMG_SIZE, IMG_SIZE))
    mask = cv2.equalizeHist(mask)
    mask = (mask > 10).astype(np.float32)
    mask = np.expand_dims(mask, axis=-1)
    return mask

def load_data(img_paths, mask_paths):
    X, y = [], []

    for img_path, mask_path in zip(img_paths, mask_paths):
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        if img.ndim == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        elif img.shape[2] == 4:
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        img = img / 255.0

        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask = preprocess_mask(mask, IMG_SIZE)
        X.append(img)
        y.append(mask)
    X = np.array(X, dtype=np.float32)
    y = np.array(y, dtype=np.float32)
    return X, y

X_train, y_train = load_data(train_imgs, train_masks)
X_val, y_val = load_data(val_imgs, val_masks)
X_test, y_test = load_data(test_imgs, test_masks)

print("Train:", X_train.shape, y_train.shape)
print("Val:", X_val.shape, y_val.shape)
print("Test:", X_test.shape, y_test.shape)


Train: (2824, 128, 128, 3) (2824, 128, 128, 1)
Val: (386, 128, 128, 3) (386, 128, 128, 1)
Test: (719, 128, 128, 3) (719, 128, 128, 1)


In [ ]:
def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true * y_pred)
    dice = (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)
    return 1 - dice

@keras.saving.register_keras_serializable()
def bce_dice_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dice = dice_loss(y_true, y_pred)
    return bce + dice

In [ ]:
model.compile(optimizer='adam', loss=bce_dice_loss, metrics=['accuracy'])

In [ ]:
callbacks_segmentation=[
    EarlyStopping(patience=5, restore_best_weights=True,monitor="val_loss"),
    ModelCheckpoint("best_model_segmentation.keras", save_best_only=True,monitor="val_accuracy")
]

In [ ]:
history_segmentation = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=16,
    epochs=15,
    callbacks = callbacks_segmentation
)

Epoch 1/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 84s 313ms/step - accuracy: 0.9586 - loss: 1.1565 - val_accuracy: 0.9911 - val_loss: 0.5102
Epoch 2/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 29s 163ms/step - accuracy: 0.9881 - loss: 0.6495 - val_accuracy: 0.9908 - val_loss: 0.5041
Epoch 3/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 29s 165ms/step - accuracy: 0.9891 - loss: 0.6160 - val_accuracy: 0.9916 - val_loss: 0.4753
Epoch 4/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 29s 166ms/step - accuracy: 0.9906 - loss: 0.5575 - val_accuracy: 0.9901 - val_loss: 0.5010
Epoch 5/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 30s 171ms/step - accuracy: 0.9921 - loss: 0.5315 - val_accuracy: 0.9919 - val_loss: 0.4548
Epoch 6/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 30s 170ms/step - accuracy: 0.9899 - loss: 0.5809 - val_accuracy: 0.9911 - val_loss: 0.4885
Epoch 7/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 30s 167ms/step - accuracy: 0.9911 - loss: 0.5434 - val_accuracy: 0.9913 - val_loss: 0.5073
Epoch 8/15
177/177 ━━━━━━━━━━━━━━━━━━━━ 30s 167ms/step - accuracy: 0.9901 - loss: 0